In [ ]:
import sys
from IPython.display import Image, display
import sympy
sys.path.append('..')   # path for local package
import restflow

# Example 1: Burgers-KPZ equation

The KPZ equation is given by:
\begin{equation*}
\partial_t\phi = \kappa \nabla^2\phi + \frac{\lambda}{2} |\nabla\phi|^2  + \eta
\end{equation*}
By switching to Fourier space, the only non-zero vertex is:
\begin{equation*}
\cal{v}_2(\bold{k_1}, \bold{k_2}) = - \frac{\lambda}{2} \bold{k_1} \cdot \bold{k_2}
\end{equation*}
The only relevant one-loop graph correcting the propagator for this example is:

In [ ]:
pil_img = Image(filename='./figures/kpz_graph.jpg')
display(pil_img)

<b><ins>1st Step:</b></ins> Define the model parameters, the vectors, the propagators and the vertex functions:

In [ ]:
# define the model
class KPZ:
    def __init__(self):
        self.alpha = 0
        self.lam, self.kap, self.D = sympy.symbols('lambda kappa D')

    def f(self,k):
        return self.kap*k**2

    def v2(self,k1,k2,q):
        return -.5*self.lam*k1*k2,1

# symbols for vectors
_q, _k, dot_kq = sympy.symbols('q k (k·q)')
ctx = restflow.Context()
ctx.add_dot_product(_q,_k,dot_kq)
# create symbolic wave vectors
k = ctx.vector(_k)
q = ctx.vector(_q)

model = KPZ()

<b><ins>2nd Step:</b></ins> Create the graph and visualizes it using `plot_graph`. Create a LaTeX file for nicer rendering using `latex_graph`:

In [ ]:
v = [restflow.Vertex() for i in range(3)]
v[0].link_vertex(v[2],angle=0.12)
v[0].link_vertex(v[1])
v[1].link_vertex(v[2])
v[1].add_outgoing()
g = restflow.Graph(v)
g.label_edges(k,[q])
g.plot_graph()
g.export_latex_graph('graph_plot')

<b><ins>3rd Step:</b></ins> Calculate the integral from the graph. Here we also change coordinates $\mathbf{k} \rightarrow \mathbf{k}+\mathbf{q}/2$:

In [ ]:
g.label_edges(.5*q+k,[q])
expr = g.convert(model)
display(expr.num/expr.den)

<b><ins>4th Step:</b></ins> Calculate the integral using `integrate`:

In [ ]:
res = restflow.integrate([expr],3,k,q)
display(res)

This integral corrects the model parameter $\kappa$. Performing the wavector integral and renormalizing it, we get:

\begin{equation*}
\psi_\kappa = \frac{D\lambda^2 K_d \Lambda^{d-2}}{\kappa^3} \frac{2-d}{4d}
\end{equation*}

# Example 2: Graph with 2 external legs

Assume we want to calculate contribution to correction of $\lambda$ of the following graph (figure (a)) for the KPZ. 

This graph represents 2 graphs (figure (b) and (c)):

In [ ]:
pil_img = Image(filename='./figures/2legs_example.jpg')
display(pil_img)

Use the same system with before but we need to define the vector $p$ and the corresponding dot products:

In [ ]:
_p, dot_pk, dot_qp = sympy.symbols('p (k·p) (q·p)')
ctx.add_dot_product(_p,_k,dot_pk)
ctx.add_dot_product(_q,_p,dot_qp)
# create symbolic wave vectors
p = ctx.vector(_p)

Create the graph:

In [ ]:
v = [restflow.Vertex() for i in range(4)]
v[0].link_vertex(v[1],angle=0.12)
v[0].link_vertex(v[2])
v[2].link_vertex(v[3])
v[3].link_vertex(v[1])
v[2].add_outgoing(angle=-0.25)
v[3].add_outgoing()

Use the method `convert_perm` to calculate all the permutations of the external momenta (figure (b) and (c)):

In [ ]:
exprs = g.convert_perm(model,k,[p,q-p])
for expr in exprs:
    display(expr.num/expr.den)

Use the function `integrate` to calculate the integrals and sum the two graphs:

In [ ]:
res = restflow.integrate(exprs,3,k,q,p)
display(res)